# <b>U.S.-Medical-Insurance-Costs


## Import libraries

In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Make a DataFrames

In [168]:
observations = pd.read_csv('observations.csv')
species_info = pd.read_csv('species_info.csv')

## <b>Exploratory Data Analysis (EDA)

### EDA for `observations`

### Data Inspection



In [169]:
observations.head()

,scientific_name,park_name,observations
0,Vicia benghalensis,Great Smoky Mountains National Park,68
1,Neovison vison,Great Smoky Mountains National Park,77
2,Prunus subcordata,Yosemite National Park,138
3,Abutilon theophrasti,Bryce National Park,84
4,Githopsis specularioides,Great Smoky Mountains National Park,85


In [170]:
observations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23296 entries, 0 to 23295
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23296 non-null  object
 1   park_name        23296 non-null  object
 2   observations     23296 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 546.1+ KB


### Missing Values and Duplicates

In [171]:
missing_values = observations.isnull().sum()
duplicates = observations.duplicated().sum()

print("missing_values: \n{ms} \n\nduplicates: {d}".format(ms = missing_values, d = duplicates))


missing_values: 
scientific_name    0
park_name          0
observations       0
dtype: int64 

duplicates: 15


### Drop Duplicates

In [186]:
observations.drop_duplicates(inplace=True)
duplicates = observations.duplicated().sum()

print("duplicates: {0}\n".format(duplicates))


# Reindex the DataFrame
observations.reset_index(drop=True, inplace=True)

observations.info()

duplicates: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23281 entries, 0 to 23280
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23281 non-null  object
 1   park_name        23281 non-null  object
 2   observations     23281 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 545.8+ KB


## Numerical summarization

In [187]:
observations.describe()

,observations
count,23281.000000
mean,142.280357
std,69.885439
min,9.000000
25%,86.000000
50%,124.000000
75%,195.000000
max,321.000000
